In [22]:
import pandas as pd
import numpy as np
import json
import os

In [23]:
json_files = []
orig_json_files = []

for file in os.listdir('analysis_reports'):
    if file.endswith('.json'):
        with open(os.path.join('analysis_reports', file), 'r') as f:
            json_files.append(json.load(f))

for file in os.listdir('orig_analysis_reports'):
    if file.endswith('.json'):
        with open(os.path.join('orig_analysis_reports', file), 'r') as f:
            orig_json_files.append(json.load(f))

len(json_files), len(orig_json_files)

(700, 700)

In [24]:
json_files[0]

{'./files/question-884-pass-1.py': [{'code': 'F401',
   'filename': './files/question-884-pass-1.py',
   'line_number': 1,
   'column_number': 1,
   'text': "'math' imported but unused",
   'physical_line': 'import math\n'}]}

In [25]:
json_files_dict = {}
orig_json_files_dict = {}

for item in json_files:
    key = list(item.keys())[0]
    json_files_dict[key] = item[key]
for item in orig_json_files:
    key =list(item.keys())[0]
    orig_json_files_dict[key] = item[key]


In [26]:
json_files_dict = {key.split('/')[-1]: value for key, value in json_files_dict.items()}
orig_json_files_dict = {key.split('/')[-1]: value for key, value in orig_json_files_dict.items()}

In [27]:
filtered_keys = [key for key, value in orig_json_files_dict.items() if value == [] and json_files_dict[key] != []]

In [28]:
len(filtered_keys)

188

In [36]:
orig_codes = {}
for filename, errors in orig_json_files_dict.items():
    for error in errors:
        code = error['code']
        if code not in orig_codes:
            orig_codes[code] = {'files': [filename], 'count': 1, 'text': error['text']}
        else:
            data = orig_codes[code]
            data['files'].append(filename)
            data['count'] += 1

In [37]:
len(orig_codes)

14

In [42]:
for code, data in orig_codes.items():
    print(code, data['text'], data['count'])

F401 'math' imported but unused 11
F841 local variable 'x' is assigned to but never used 20
E501 line too long (110 > 79 characters) 23
E999 SyntaxError: invalid syntax 56
E741 ambiguous variable name 'l' 56
F821 undefined name 'd2' 34
E712 comparison to False should be 'if cond is False:' or 'if not cond:' 27
E129 visually indented line with same indent as next logical line 1
E251 unexpected spaces around keyword / parameter equals 10
E221 multiple spaces before operator 5
E502 the backslash is redundant between brackets 1
F403 'from array import *' used; unable to detect undefined names 2
F405 'ceil' may be undefined, or defined from star imports: array, math 1
F823 local variable 'sum' defined as a builtin referenced before assignment 1


In [39]:
codes = {}
for filename, errors in json_files_dict.items():
    for error in errors:
        code = error['code']
        if code not in codes:
            codes[code] = {'files': [filename], 'count': 1, 'text': error['text']}
        else:
            data = codes[code]
            data['files'].append(filename)
            data['count'] += 1

In [40]:
len(codes)

17

In [52]:
filenames = set()
for code, data in codes.items():
    if data['count'] > 50 and ( code not in orig_codes or orig_codes[code]['count'] < 50):
        print(code, data['text'], data['count'])
        for f in data['files']:
            filenames.add(f)

F401 'math' imported but unused 96
E501 line too long (224 > 79 characters) 90
F821 undefined name 'arr' 53
E225 missing whitespace around operator 68


Codes to consider for fine tuning - F401, E501, F821

In [53]:
len(filenames)

198

In [54]:
for f in filtered_keys:
    filenames.add(f)

len(filenames)

240

In [66]:
response_codes = {}
for filename in filenames:
    with open('files/' + filename, 'r') as file:
        data = file.read()
        id = int(filename.split('-')[1])
        response_codes[id] = data

In [67]:
orig_codes = {}
for filename in filenames:
    with open('orig_files/' + filename, 'r') as file:
        data = file.read()
        id = int(filename.split('-')[1])
        orig_codes[id] = data

In [68]:
len(response_codes), len(orig_codes)

(240, 240)

In [70]:
print(response_codes[756])

from bisect import bisect_left


def smallestPermutation(arr, N):
    # Create a list of indices from 0 to N-1
    indices = list(range(N))

    # Sort the indices based on the value of arr at that index
    indices.sort(key=lambda i: arr[i])

    # Return the sorted indices
    return indices



In [71]:
response_df = pd.read_csv('./train_response_300_1000.csv')
response_df.head()

,id,text,code,question,functions,code_context,context,llama_prompt,response
0,302,"Sum of the first N terms of the series 5 , 12 ...",def calculateSum(n):\n return (2 * (n * (n ...,"Sum of the first N terms of the series 5 , 12 ...",['def calculateSum(n):'],def calculateSum(n):,def calculateSum(n):,<s>[INST]<<SYS>>\nProvide answers in Python. W...,<s>[INST]<<SYS>>\nProvide answers in Python. W...
1,303,Program to find number of solutions in Quadrat...,"def checkSolution(a, b, c):\n if ((b * b) -...",Program to find number of solutions in Quadrat...,"['def checkSolution(a, b, c):']","def checkSolution(a, b, c):","def checkSolution(a, b, c):",<s>[INST]<<SYS>>\nProvide answers in Python. W...,<s>[INST]<<SYS>>\nProvide answers in Python. W...
2,304,Program to convert KiloBytes to Bytes and Bits...,def Bits(kilobytes):\n Bits = kilobytes * 8...,Program to convert KiloBytes to Bytes and Bits.,"['def Bits(kilobytes):', 'def Bytes(kilobytes):']",def Bits(kilobytes):,def Bits(kilobytes):,<s>[INST]<<SYS>>\nProvide answers in Python. W...,<s>[INST]<<SYS>>\nProvide answers in Python. W...
3,306,Find sum of the series ? 3 + ? 12 + ... ... .....,import math\n\n\ndef findSum(n):\n return m...,Find sum of the series ? 3 + ? 12 + ... ... .....,['def findSum(n):'],import math\n\n\ndef findSum(n):,import math\n\n\ndef findSum(n):,<s>[INST]<<SYS>>\nProvide answers in Python. W...,<s>[INST]<<SYS>>\nProvide answers in Python. W...
4,307,Find the sum of the series x ( x + y ) + x ^ 2...,"def sum(x, y, n):\n sum1 = ((x ** 2) * (x *...",Find the sum of the series x ( x + y ) + x ^ 2...,"['def sum(x, y, n):']","def sum(x, y, n):","def sum(x, y, n):",<s>[INST]<<SYS>>\nProvide answers in Python. W...,<s>[INST]<<SYS>>\nProvide answers in Python. W...


In [74]:
df = pd.DataFrame({'id': response_df['id'], 'prompt': response_df['llama_prompt']})
df = df[df['id'].isin(response_codes.keys())]

df['accepted_code'] = df['id'].map(lambda key: orig_codes[key])
df['rejected_code'] = df['id'].map(lambda key: response_codes[key])
df.head()

,id,prompt,accepted_code,rejected_code
3,306,<s>[INST]<<SYS>>\nProvide answers in Python. W...,import math\n\n\ndef findSum(n):\n return m...,import math\n\n\ndef findSum(n):\n return (...
4,307,<s>[INST]<<SYS>>\nProvide answers in Python. W...,"def sum(x, y, n):\n sum1 = ((x ** 2) * (x *...","def sum(x, y, n):\n return sum([x * (x + y)..."
10,313,<s>[INST]<<SYS>>\nProvide answers in Python. W...,dp = [[-1 for j in range(500)]for i in range(5...,dp = [[-1 for j in range(500)]for i in range(5...
16,319,<s>[INST]<<SYS>>\nProvide answers in Python. W...,import sys\n\n\ndef maximumOccurrence(s):\n ...,import sys\n\n\ndef maximumOccurrence(s):\n ...
17,320,<s>[INST]<<SYS>>\nProvide answers in Python. W...,mod = 1000000007\ndp = [[[-1 for i in range(2)...,mod = 1000000007\ndp = [[[-1 for i in range(2)...
